In [41]:
import nltk 
nltk.download('twitter_samples') # dataset 
nltk.download('punkt') #module is a pre-trained model that helps you tokenize words and sentences 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# stopwords package like 'a,the,an'
nltk.download('stopwords')


[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/devil/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to /Users/devil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/devil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/devil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/devil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
import re, string
from nltk.corpus import stopwords
from nltk import FreqDist
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize


In [56]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')


In [57]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

# print(tweet_tokens[0])


In [58]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')


In [59]:

stop_words = stopwords.words('english')


def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [60]:

# print(remove_noise(tweet_tokens[0], stop_words))
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [61]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)


In [62]:

freq_dist_pos = FreqDist(all_pos_words)
# print(freq_dist_pos.most_common(10))

In [63]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [64]:


positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [65]:
#using naive bayes classifier


classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2088.7 : 1.0
                      :) = True           Positi : Negati =   1629.3 : 1.0
                     sad = True           Negati : Positi =     33.6 : 1.0
                    glad = True           Positi : Negati =     20.0 : 1.0
                  arrive = True           Positi : Negati =     19.8 : 1.0
                     x15 = True           Negati : Positi =     18.0 : 1.0
                 awesome = True           Positi : Negati =     17.3 : 1.0
               community = True           Positi : Negati =     14.1 : 1.0
                 welcome = True           Positi : Negati =     12.1 : 1.0
                     ugh = True           Negati : Positi =     11.9 : 1.0
None


In [66]:

custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
t = word_tokenize(custom_tweet)
custom_tokens = remove_noise(t)

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative
